In [48]:
import numpy as np
import pandas as pd
import keras
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.neighbors import DistanceMetric

In [2]:
canto = pd.read_csv('./canto-concensus-lomax-normalized.csv').to_numpy()
triplet = pd.read_csv('./triplet.csv').to_numpy()

In [3]:
X = []
Y = []

In [4]:
for i in range(6): # Groups
    for j in range(5):
        for k in range(5):
            a = np.array(canto[i * 5 + j][1:-1])
            b = np.array(canto[i * 5 + k][1:-1])
            X.append(np.concatenate([a, b]))
            Y.append(triplet[i * 5 + j, i * 5 + k + 1] / 100.0)

In [5]:
X = np.array(X)
Y = np.array(Y)

print(X.shape)
print(Y.shape)

(150, 72)
(150,)


In [30]:
a = Input(shape=(72,))
x = Dense(256, activation='relu')(a)
x = Dense(128, activation='relu')(x)
b = Dense(1, activation='sigmoid')(x)
model = Model(inputs=a, outputs=b)

In [31]:
model.compile(loss='mean_squared_error', optimizer='Adam')

In [38]:
model.fit(
    x=X,
    y=Y,
    epochs=8192,
    shuffle=True,
)

996e-07
Epoch 7975/8192
150/150 [==============================] - 0s 89us/step - loss: 4.1903e-07
Epoch 7976/8192
150/150 [==============================] - 0s 120us/step - loss: 4.1788e-07
Epoch 7977/8192
150/150 [==============================] - 0s 87us/step - loss: 4.1006e-07
Epoch 7978/8192
150/150 [==============================] - 0s 99us/step - loss: 4.0956e-07
Epoch 7979/8192
150/150 [==============================] - 0s 72us/step - loss: 4.0460e-07
Epoch 7980/8192
150/150 [==============================] - 0s 81us/step - loss: 4.0232e-07
Epoch 7981/8192
150/150 [==============================] - 0s 77us/step - loss: 3.9887e-07
Epoch 7982/8192
150/150 [==============================] - 0s 85us/step - loss: 3.9503e-07
Epoch 7983/8192
150/150 [==============================] - 0s 133us/step - loss: 3.9216e-07
Epoch 7984/8192
150/150 [==============================] - 0s 73us/step - loss: 3.8862e-07
Epoch 7985/8192
150/150 [==============================] - 0s 72us/step - loss: 

In [39]:
result = np.zeros((30, 30))

In [40]:
for i in range(30):
    for j in range(30):
        a = np.array(canto[i][1:-1])
        b = np.array(canto[j][1:-1])
        similarity = model.predict(np.array([np.concatenate([a, b])]))
        result[i, j] = similarity

In [41]:
result

array([[1.20951154e-06, 3.93120706e-01, 5.44879198e-01, 4.83884037e-01,
        8.47608268e-01, 6.34938955e-01, 7.05483416e-03, 6.34528339e-01,
        6.18831754e-01, 7.55206048e-01, 3.10552716e-01, 6.40201569e-01,
        9.15439334e-04, 6.67522967e-01, 8.16442192e-01, 5.75962543e-01,
        6.49629772e-01, 6.66487683e-03, 3.39563996e-01, 5.21538377e-01,
        6.43227756e-01, 3.98479253e-01, 8.53008777e-03, 8.70949763e-04,
        7.28351891e-01, 7.46594548e-01, 1.60686875e-04, 6.14684880e-01,
        4.47550446e-01, 6.44821107e-01],
       [3.94128054e-01, 3.21024010e-04, 7.87885189e-01, 5.15181899e-01,
        7.25750804e-01, 6.57350123e-01, 5.86322784e-01, 8.76656532e-01,
        6.18574381e-01, 3.51152301e-01, 5.71161211e-01, 5.45773208e-01,
        4.00000393e-01, 5.74601889e-01, 8.72094035e-01, 6.74384475e-01,
        2.83774585e-01, 6.41194701e-01, 5.11787295e-01, 2.45628089e-01,
        6.59172356e-01, 3.39259744e-01, 5.50562441e-01, 3.75378251e-01,
        6.43610597e-01,

In [47]:
meaned = np.zeros((30, 30))
for i in range(30):
    for j in range(30):
        meaned[i, j] = (result[i, j] + result[j, i]) / 2

np.savetxt('result.csv', np.asarray(meaned), delimiter=',', fmt="%.2f")

In [54]:
dist = DistanceMetric.get_metric('euclidean')
dist_x = []
for i in range(30):
    dist_x.append(np.array(canto[i][1:-1]))
res = dist.pairwise(dist_x)

np.savetxt('canto-distance.csv', np.array(res) / np.array(res).max(), delimiter=',', fmt="%.2f")